<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/NER_name_gender_ethnicity_redact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q
!pip install langchain -q
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!pip install name2nat
!pip install nameCreator


In [78]:
!pip install --upgrade name2nat
!pip install --upgrade urllib3
!pip install --upgrade torch torchvision

  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1


  Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.34.48 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.2.1 which is incompatible.
flair 0.13.1 requires urllib3<2.0.0,>=1.0.0, but you have urllib3 2.2.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 883.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 2.4 MB

Design a MVP for masking Name and synthesizing Name according to his/her gender and enthinicity.
    #1. Streamlit for UI (same as Presidio)
    #2. Langchain for combining models, and other packages

Streamlit setup

In [10]:
!wget -q -O - ipv4.icanhazip.com

34.42.58.1


In [ ]:
# Bring in deps
import os
import streamlit as st
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper
from langchain.llms import HuggingFaceModel

#for the model, use Hugging face pipeline
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

#name-enthicity and enthicity-name
from name2nat import Name2nat
import nameCreator

In [11]:
# App framework
# save it as app.py
st.title('🦜🔗 Name Entity Masking')
prompt = st.text_input('Plug in your prompt here')

In [17]:
# run the app, copy url and paste the ip we got to get access to the streamlit webpage.
!streamlit run app.py & npx localtunnel --port 8501

[..................] | fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.42.58.1:8501

npx: installed 22 in 2.388s
your url is: https://curly-beds-win.loca.lt
  Stopping...
^C


Create Langchain and call the BERT for NER task

In [22]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.c

In [54]:
#Get the NER BERT model from Hugging face
#use default tokenizer and
ner_pipeline = pipeline("ner", model="bigcode/starpii")
text = "My name is Ryan Chen"
entities = ner_pipeline(text)
print(entities)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-NAME', 'score': 0.999767, 'index': 3, 'word': 'ĠR', 'start': 10, 'end': 12}, {'entity': 'I-NAME', 'score': 0.9997906, 'index': 4, 'word': 'yan', 'start': 12, 'end': 15}, {'entity': 'I-NAME', 'score': 0.99888223, 'index': 5, 'word': 'ĠChen', 'start': 15, 'end': 20}]


In [86]:
# name-gender from genderize.io using its API
import requests, json

name = 'Ryan'
content = requests.get(f"https://api.genderize.io/?name={name}").text
gender = json.loads(content)['gender']

print(gender)

male


In [115]:
#Name generator
#upload male's and female's name and firstF.py, firstM.py
#female's name .txt files are in f_names folder
from firstF import FirstNameFemale
from firstM import FirstNameMale

FirstNameMale('american')

'Bellamy'

In [97]:
!pip install racebert

In [105]:
from racebert import RaceBERT

model = RaceBERT()

# To predict race
model.predict_race("Ryan Chen")


# Race	Label
# Non-hispanic White	nh_white
# Hispanic	hispanic
# Non-hispanic Black	nh_black
# Asian & Pacific Islander	api
# American Indian & Alaskan Native	aian


[{'label': 'api', 'score': 0.8302366137504578}]

In [111]:
#Get the ethnicity BERT model from Hugging face
ethnicity_pipeline = pipeline("text-classification", model="pparasurama/raceBERT-ethnicity")
name = "Ryan Chen"
result = ethnicity_pipeline(name)
print(result)

[{'label': 'Asian,GreaterEastAsian,EastAsian', 'score': 0.926744818687439}]


In [ ]:
#input prompt my name is Ryan Chen
# NER BERT: my name is {Name}
# Gender genderize API: male
# Ethnicity BERT: Asian
# either redact it as Asian male or other Name.

In [ ]:
# design a function for given POS and get NER to my name is B-name, I-name
# Langchain with BERT? do I need it here? forget about it. No need